# Étude d'une équation de convection scalaire

`Python` sera utilisé ici comme `matlab`. Des fonctionnalités supplémentaires peuvent être ajoutées par l'import de modules, standards à une distribution (comme `math`, `numpy`) ou personnalisés comme ci-dessous. Des fonctionnalités d'édition sont propres à [`Ipython/Notebook`](#ipython).

In [ ]:
%matplotlib inline
import numpy             as np
import numpy.linalg      as alg
import matplotlib.pyplot as plt
import time
#
import flowdyn.mesh  as mesh
import flowdyn.modelphy.euler as euler
import flowdyn.modeldisc      as modeldisc
#import flowdyn.field as data
from flowdyn.xnum        import *
from flowdyn.integration import *

On cherche à résoudre l'évolution instationnaire du problème linéaire de convection suivant

$$ \frac{\partial q}{\partial t} + a \frac{\partial q}{\partial x} = 0 $$

pour la quantité transportée $q(x,t)$ et la condition intiale $q_0(x)$ sur le domaine $[0;\ell]$ avec des conditions périodiques. On choisit $\ell=1\rm~m$ et $a=1\rm~m/s$. 


## Définition des maillages, du modèle physique et solution initiales


In [ ]:
mesh400 = mesh.unimesh(ncell=400, length=1.)
mesh200 = mesh.unimesh(ncell=200, length=1.)
mesh100 = mesh.unimesh(ncell=100, length=1.)
mesh50  = mesh.unimesh(ncell=50,  length=1.)
mesh10  = mesh.unimesh(ncell=10,  length=1.)
mgmesh  = mesh.refinedmesh(ncell=100, length=1., ratio=2.)
momesh  = mesh.morphedmesh(ncell=100, length=1., morph=lambda x:((x-.5)/.5)**3 *.5 +.5)

mymodel = euler.model()
print(mymodel.list_bc())

#plt.plot(momesh.centers())


## Calcul et comparaison de spectre


1. le spectre dépend-t-il de la condition initiale ? pourquoi ?
2. qu'attend-t-on de la partie droite du spectre ? 
3. qu'attend-t-on de la partie gauche du spectre et quel est son impact ?

In [ ]:
mach  = 0.05
rttot = (1.+.2*mach**2) / 1.4
ptot  = (1.+.2*mach**2)**3.5
lmesh = momesh
num   = extrapol3()
bcp = { 'type': 'per' }
bcL = { 'type': 'insub',  'ptot': ptot, 'rttot': rttot }
bcR = { 'type': 'outsub', 'p': 1. }

#rhs = modeldisc.fvm(model, meshsim, muscl(vanalbada), 
#      bcL=bcL, bcR=bcR)
#solver = rk3ssp(meshsim, rhs)

# extrapol1(), extrapol2()=extrapolk(1), centered=extrapolk(-1), extrapol3=extrapol(1./3.) 
# explicit, rk2, rk3ssp, rk4, implicit, trapezoidal=cranknicolson

rhs  = modeldisc.fvm(mymodel, lmesh, num, bcp, bcp)

finit = rhs.fdata(mymodel.prim2cons([  1.4, mach, 1. ])) # rho, u, p

solver = implicit(lmesh, rhs)
jac    = solver.calc_jacobian(finit, epsdiff=1.e-3)
val, vec = alg.eig(jac)
results = val/lmesh.ncell

# display and save results to png file
style=['o', 'x', 'D', '*', 'o', 'o']
fig, ax = plt.subplots(1, 2, figsize=(14,6))
ax[0].scatter(results.real, results.imag, marker=style[0])
ax[1].scatter(results.real, results.imag, marker=style[0])
ax[1].set_xlim(-.1, .05)
ax[1].set_ylim(-.1, .5)
#plt.legend(labels, loc='upper left',prop={'size':10})

In [ ]:
np.savetxt("eigvals.txt", np.array([results.real, results.imag]).transpose(), newline='\n')
#np.savetxt("eigvals.txt", (np.arange(10), 2*np.arange(10)), newline='\n')
#np.array([results.real, results.imag]).transpose()

In [ ]:
from IPython.core.display import HTML ; HTML(open("./custom.css", "r").read()) # notebook style